# Data Mining / Prospeção de Dados


# Project - Classification/Regression

## Dataset and Tools

In this project you should use [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org) and **[Scikit-learn](http://scikit-learn.org/stable/). You are also allowed to use [Orange3](https://orange.biolab.si).**

The dataset to be analysed is **`RestaurantsRevenue.csv`**, a modified version of the test dataset used in Kaggle's competition ["Restaurant Revenue Prediction"](https://www.kaggle.com/c/restaurant-revenue-prediction/overview). 

**This project challenges you twice** by asking you to tackle a
1. **Regression Task**: predict the revenue, and a
2. **Classification Task**: predict a revenue category.

The available variables are:

* **Id :** Restaurant id. 
* **Open Date :** opening date for a restaurant
* **City :** City that the restaurant is in. Note that there are unicode in the names. 
* **City Group:** Type of the city. Big cities, or Other. 
* **Type:** Type of the restaurant. FC: Food Court, IL: Inline, DT: Drive Thru, MB: Mobile
* **P1, P2 - P37:** There are three categories of these obfuscated data. Demographic data are gathered from third party providers with GIS systems. These include population in any given area, age and gender distribution, development scales. Real estate data mainly relate to the m2 of the location, front facade of the location, car park availability. Commercial data mainly include the existence of points of interest including schools, banks, other QSR operators.


The targets are:
1. **`Revenue`:** The revenue column indicates a (transformed) revenue of the restaurant in a given year and is the target of predictive analysis. Please note that the values are transformed so they don't mean real dollar values. 
2. **`RevenueCategory`** - the revenue category, where price can be below 12000 ("<12K"), between 12000 and 20000 ("12K-20K"), or above 20000 (">20K"). This is the target variable that you're trying to predict in the classification task. 

## Team Identification

**GROUP 010**

Students:

* Student 1 - Hugo Rocha Nº48331
* Student 2 - Leonardo Ricardo Nº55047
* Student 3 - Filipa Santos Nº57167

## 1. Load and Preprocess Dataset

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import VotingClassifier, VotingRegressor

  > - Before going through the 3 tasks above, we will explore the dataset

In [4]:
df = pd.read_csv('RestaurantsReveneu.csv')

In [5]:
df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,10033.0
1,1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,...,3.0,0,0,0,0,0,0,0,0,9355.0
2,2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,3.0,0,0,0,0,0,0,0,0,11353.0
3,3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,3.0,0,4,0,0,0,0,0,0,10828.0
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,9354.0


In [6]:
df.shape

(100000, 43)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 43 columns):
Id            100000 non-null int64
Open Date     100000 non-null object
City          100000 non-null object
City Group    100000 non-null object
Type          100000 non-null object
P1            100000 non-null int64
P2            100000 non-null float64
P3            100000 non-null float64
P4            100000 non-null float64
P5            100000 non-null int64
P6            100000 non-null int64
P7            100000 non-null int64
P8            100000 non-null int64
P9            100000 non-null int64
P10           100000 non-null int64
P11           100000 non-null int64
P12           100000 non-null int64
P13           100000 non-null float64
P14           100000 non-null int64
P15           100000 non-null int64
P16           100000 non-null int64
P17           100000 non-null int64
P18           100000 non-null int64
P19           100000 non-null int64
P20           

In [8]:
df.describe() #checking descriptive statistics of the numerical variables

,Id,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.00000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,49999.500000,4.088030,4.428085,4.215325,4.396025,1.989590,2.881900,5.30051,4.93100,5.251380,...,3.084000,2.083300,1.193330,1.942640,0.987430,2.108670,1.832830,1.968890,0.973500,14698.061620
std,28867.657797,2.812963,1.428865,0.842161,1.035827,1.065314,1.531429,2.17858,1.71849,1.702632,...,1.783927,4.309479,2.307944,3.971298,1.534808,4.685414,3.228769,3.805773,1.677267,6705.081965
min,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,1.00000,1.00000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6271.000000
25%,24999.750000,2.000000,3.750000,4.000000,4.000000,1.000000,2.000000,5.00000,4.00000,4.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10143.000000
50%,49999.500000,3.000000,5.000000,4.000000,4.000000,2.000000,2.000000,5.00000,5.00000,5.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12951.000000
75%,74999.250000,4.000000,5.000000,4.000000,5.000000,2.000000,4.000000,5.00000,5.00000,5.000000,...,3.000000,3.000000,1.000000,3.000000,2.000000,3.000000,4.000000,3.000000,2.000000,16923.000000
max,99999.000000,15.000000,7.500000,6.000000,7.500000,6.000000,10.000000,10.00000,10.00000,10.000000,...,10.000000,25.000000,15.000000,25.000000,6.000000,30.000000,15.000000,20.000000,8.000000,52294.000000


In [9]:
df['Open Date'] = pd.to_datetime(df['Open Date'])

In [10]:
df['Open Date'].nunique() #checking unique dates 

310

In [11]:
df['Open Date']=df['Open Date'].dt.strftime('%m-%Y')

In [12]:
#converting to just %m-%Y format since we don't consider the day to be relevant
df['Open Date']

0        01-2011
1        03-2011
2        10-2013
3        05-2013
4        07-2013
          ...   
99995    01-2000
99996    07-2011
99997    12-2012
99998    10-2013
99999    10-2010
Name: Open Date, Length: 100000, dtype: object

In [13]:
first_date = df["Open Date"].min()
last_date = df["Open Date"].max()

In [14]:
first_date

'01-1999'

In [15]:
last_date

'12-2013'

In [16]:
df.City.value_counts() #checking the different cities in turkey

İstanbul          34087
Ankara             8720
İzmir              6465
Antalya            5911
Kocaeli            4364
Mersin             2735
Adana              2514
Balıkesir          2463
Bursa              2441
Muğla              1823
Aydın              1617
Tekirdağ           1577
Konya              1576
Gaziantep          1487
Edirne             1230
Manisa             1227
Çanakkale           965
Denizli             964
Diyarbakır          954
Hatay               951
Zonguldak           926
Eskişehir           900
Trabzon             660
Aksaray             650
Bolu                631
Yalova              630
Kırıkkale           622
Malatya             616
Mardin              610
Şanlıurfa           609
Sakarya             604
Batman              604
Rize                345
Artvin              344
Bilecik             339
Afyonkarahisar      331
Nevşehir            328
Sivas               326
Samsun              324
Kayseri             323
Kırşehir            319
Erzincan        

In [17]:
df['City Group'].value_counts()

Other         50728
Big Cities    49272
Name: City Group, dtype: int64

In [18]:
df['Type'].value_counts()

FC    57019
IL    40447
DT     2244
MB      290
Name: Type, dtype: int64

  > - **1.1 Missing Values**

In [19]:
df.isna().sum() 

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
dtype: int64

In [20]:
#we will take a closer look at columns starting with "P" which represent obfuscated data
for col in df.columns:
    if col[0] == 'P':
        print (col, '- unique values:', len(df[col].unique()))

P1 - unique values: 9
P2 - unique values: 9
P3 - unique values: 7
P4 - unique values: 7
P5 - unique values: 6
P6 - unique values: 8
P7 - unique values: 7
P8 - unique values: 8
P9 - unique values: 5
P10 - unique values: 4
P11 - unique values: 8
P12 - unique values: 7
P13 - unique values: 5
P14 - unique values: 10
P15 - unique values: 9
P16 - unique values: 9
P17 - unique values: 10
P18 - unique values: 9
P19 - unique values: 9
P20 - unique values: 9
P21 - unique values: 9
P22 - unique values: 5
P23 - unique values: 9
P24 - unique values: 9
P25 - unique values: 9
P26 - unique values: 10
P27 - unique values: 10
P28 - unique values: 9
P29 - unique values: 8
P30 - unique values: 10
P31 - unique values: 10
P32 - unique values: 10
P33 - unique values: 7
P34 - unique values: 11
P35 - unique values: 7
P36 - unique values: 10
P37 - unique values: 8


In [21]:
obfuscated_cols = []
for col in df.columns:
    if col[0] == 'P':
        obfuscated_cols.append(col)

In [22]:
(~df[obfuscated_cols].all(1)).sum() 
#out of 100000 rows, 92414 have at least one zero in the obfuscated features P1-037

92414

In [23]:
df_ob = df[obfuscated_cols] #getting a dataframe with just the features starting with "P"

In [24]:
missing_obfuscated_cols = df_ob[df_ob == 0].count(axis=0)/df_ob.shape[0]*100

In [25]:
missing_obfuscated_cols.sort_values() #getting the missing values (when an instance is zero) for each of the "P" features

P1      0.000
P28     0.000
P23     0.000
P22     0.000
P21     0.000
P20     0.000
P13     0.000
P12     0.000
P11     0.000
P19     0.000
P9      0.000
P8      0.000
P7      0.000
P2      0.000
P6      0.000
P5      0.000
P4      0.000
P10     0.000
P3      0.318
P29     3.083
P31    65.566
P30    65.596
P36    65.662
P14    65.734
P25    65.738
P24    65.766
P15    65.772
P35    65.776
P26    65.784
P32    65.787
P33    65.791
P17    65.792
P34    65.832
P18    65.980
P37    66.029
P16    66.094
P27    66.193
dtype: float64

In [26]:
#dropping features with more than 65% of missing values
missing_obfuscated_cols_to_drop = missing_obfuscated_cols[missing_obfuscated_cols>65].index
missing_obfuscated_cols_to_drop = list(missing_obfuscated_cols_to_drop)
missing_obfuscated_cols_to_drop

['P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P24',
 'P25',
 'P26',
 'P27',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35',
 'P36',
 'P37']

In [27]:
df = df.drop(missing_obfuscated_cols_to_drop, axis=1)

In [28]:
#replacing missing values of P3 and P29 with the mean
df['P3']=df['P3'].replace(0,df['P3'].mean())
df['P29']=df['P3'].replace(0,df['P29'].mean())

  > - **P1, P2 - P37**: There are three categories of these obfuscated data:
    - Demographic data (population in any given area, age and gender distribution, development scales)
    - Real estate data (m2 of the location, front facade of the location, car park availability)
    - Commercial data (points of interest including schools, banks, other QSR operators)

  > - Although we know that there are 3 categories in the obfuscated data represented from columns P1-P37, we are not able to choose an imputation method for those with high missing values (>65%) at the risk of changing the original data. A *zero* could represent that, for example, there are no points of interest in a given city or simply they were not measured. We have chosen to drop from our analysis features whose missing values represent more than 65%, those are: P31, P30, P36, P14, P25, P24, P15, P35, P26, P32, P33, P17, P34, P18, P37, P16 and P27. As such, we will end up with only 2 of the obfuscated features with missing values, **P3 and P29**, with zeros representing 0.3% and 3.1%. Given these low percetages, we will **replace the zeros with the mean**. 

  > - One call out about these obfuscated features, whose instances seem to between 1 and 25 could be that, for example, they could be the order of the answers in a formulaire e.g. for question 1 (P1), Restaurant with ID 1 has answered the first option 1. We should again, keep in mind, we are just making this assumption. 
This would highlight the fact that this representation may cause some issues since some Machine Learning algorithms will assume that two nearby values (answering option 1 and option 2) are more similar than two distant (answering option 1 and option 25). It may not be the case for the obfuscated features. To fix this, we will create **binary attributes per obfuscated feature** e.g. one attribute equal to 1 when P1 is "1" (and 0 otherwise). We will additionally use **One Hot Encoding for the categorical features** we have analysed: City, City Group and Type as well as the time series data (Open Date). 

  > - **1.2 Feature selection**
 
  > - **1.2.a) Categorical Features**

  > - As mentioned in *1.1* we will create dummy varibles for the features in the dataset.

In [29]:
df_cat = df.drop(['Id','revenue'],axis=1) #dropping the unique identifier column and the target column

In [30]:
cat_encoder = OneHotEncoder()
df_cat_1hot = cat_encoder.fit_transform(df_cat)

In [31]:
df_cat_1hot.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
df_cat_1hot.shape #sanity check, we have now 350 new columns

(100000, 350)

In [33]:
cat_encoder.get_feature_names()

array(['x0_01-1999', 'x0_01-2000', 'x0_01-2005', 'x0_01-2009',
       'x0_01-2011', 'x0_01-2012', 'x0_01-2013', 'x0_01-2014',
       'x0_02-1998', 'x0_02-2000', 'x0_02-2004', 'x0_02-2007',
       'x0_02-2009', 'x0_02-2010', 'x0_02-2011', 'x0_02-2012',
       'x0_02-2013', 'x0_03-1996', 'x0_03-1998', 'x0_03-2002',
       'x0_03-2005', 'x0_03-2006', 'x0_03-2007', 'x0_03-2008',
       'x0_03-2009', 'x0_03-2010', 'x0_03-2011', 'x0_03-2012',
       'x0_03-2013', 'x0_04-1997', 'x0_04-1998', 'x0_04-2000',
       'x0_04-2006', 'x0_04-2007', 'x0_04-2008', 'x0_04-2009',
       'x0_04-2010', 'x0_04-2011', 'x0_04-2012', 'x0_04-2013',
       'x0_05-1995', 'x0_05-1997', 'x0_05-1998', 'x0_05-2006',
       'x0_05-2007', 'x0_05-2008', 'x0_05-2009', 'x0_05-2010',
       'x0_05-2011', 'x0_05-2012', 'x0_05-2013', 'x0_06-1996',
       'x0_06-1997', 'x0_06-1999', 'x0_06-2000', 'x0_06-2001',
       'x0_06-2003', 'x0_06-2006', 'x0_06-2007', 'x0_06-2009',
       'x0_06-2010', 'x0_06-2011', 'x0_06-2012', 'x0_06

In [34]:
X = pd.DataFrame(df_cat_1hot.toarray(), columns=cat_encoder.get_feature_names())
#getting the dataframe with the one hot encoded features

In [35]:
X.head(2)

,x0_01-1999,x0_01-2000,x0_01-2005,x0_01-2009,x0_01-2011,x0_01-2012,x0_01-2013,x0_01-2014,x0_02-1998,x0_02-2000,...,x22_7.5,x22_10.0,x22_12.5,x23_2.0,x23_3.0,x23_4.0,x23_4.215325,x23_4.5,x23_5.0,x23_6.0
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


  > - **1.3 Class Imbalance**

In [36]:
df.revenue.hist(); #visually checking the distribution of 'revenue'

In [37]:
df.revenue.max() #to get the last value for the bin numbers

52294.0

In [38]:
#creating the new feature for the classification task, with 3 different classes
df['revenue_category'] = pd.cut(df.revenue,
                               bins=[0, 12000, 20000, 52295], 
                                right=False,
                                labels = ['<12K','12K-20K','>20K'])

In [39]:
df.loc[df.revenue==12000][['revenue','revenue_category']].head(2) #sanity check that revenue=12000 belongs to class 12K-20K

,revenue,revenue_category
1389,12000.0,12K-20K
1471,12000.0,12K-20K


In [40]:
df.loc[df.revenue==df.revenue.max()][['revenue','revenue_category']] #sanity check

,revenue,revenue_category
74427,52294.0,>20K


In [41]:
#checking class proportions across the whole dataset
pd.DataFrame(np.round(df['revenue_category'].value_counts(normalize=True) * 100,2))

,revenue_category
12K-20K,43.94
<12K,43.40
>20K,12.66


In [42]:
df.loc[df['revenue_category'].isnull().values] #checking for any null values on the df again

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P13,P19,P20,P21,P22,P23,P28,P29,revenue,revenue_category


In [43]:
y =df['revenue_category'] #definiting the target variable

  > - For the Classification task we will select the target feature **'revenue_category'.**
  We have applied the **stratified sampling method** to ensure that the right number of instances are sampled from each of the 3 categories (<12K, 12K-20K, >20K) such that the test set is **representative of the overall population** as we can confirm below.

In [44]:
X_train, X_test, y_train_cat, y_test_cat = train_test_split(X, df['revenue_category'], test_size = 0.2, stratify=df['revenue_category'])

In [45]:
print("X_train shape is : ", X_train.shape)
print("X_test shape  is : ", X_test.shape)
print("y_train shape is : ", y_train_cat.shape)
print("y_test shape is : ", y_test_cat.shape)

X_train shape is :  (80000, 350)
X_test shape  is :  (20000, 350)
y_train shape is :  (80000,)
y_test shape is :  (20000,)


In [46]:
pd.DataFrame(np.round(df['revenue_category'].value_counts(normalize=True) * 100,2)) #overall population proportion of the classes 

,revenue_category
12K-20K,43.94
<12K,43.40
>20K,12.66


In [47]:
pd.DataFrame(np.round(y_test_cat.value_counts(normalize=True) * 100,2)) #after splitting into train and test
#The test set generated using stratified sampling has revenue category proportions similar to those in the full dataset we've previously seen above

,revenue_category
12K-20K,43.94
<12K,43.40
>20K,12.66


  > - Due to the high number of features present, we will apply the **PCA** to both the classification and regression datasets.

In [48]:
pca = PCA()

pca.fit(X_train)

cumsum = np.cumsum(pca.explained_variance_ratio_)

dimension_chosen = np.argmax(cumsum>=0.80) + 1
dimension_chosen #we're reducing the dimensions to just 57 while keeping 80% of the explained variance of the original dataset

57

In [49]:
pca = PCA(n_components = dimension_chosen)

In [50]:
X_train_cat_pca = pca.fit_transform(X_train)
X_test_cat_pca = pca.fit_transform(X_test)

In [51]:
print("X_train_cat_pca shape is : ", X_train_cat_pca.shape)
print("X_test_cat_pca shape  is : ", X_test_cat_pca.shape)
print("y_train shape is : ", y_train_cat.shape)
print("y_test shape is : ", y_test_cat.shape)

X_train_cat_pca shape is :  (80000, 57)
X_test_cat_pca shape  is :  (20000, 57)
y_train shape is :  (80000,)
y_test shape is :  (20000,)


  > - For the Regression task we will select the target feature **'revenue'** and apply **PCA** similarly.

In [52]:
X_train_rev, X_test_rev, y_train_rev, y_test_rev = train_test_split(X, df['revenue'], test_size = 0.2)

In [53]:
print("X_train_rev shape is : ", X_train_rev.shape)
print("X_test_rev shape  is : ", X_test_rev.shape)
print("y_train_rev shape is : ", y_train_rev.shape)
print("y_test_rev shape is : ", y_test_rev.shape)

X_train_rev shape is :  (80000, 350)
X_test_rev shape  is :  (20000, 350)
y_train_rev shape is :  (80000,)
y_test_rev shape is :  (20000,)


In [54]:
pca = PCA()

pca.fit(X_train_rev)

cumsum = np.cumsum(pca.explained_variance_ratio_)

dimension_chosen_rev = np.argmax(cumsum>=0.80) + 1
dimension_chosen_rev 

56

In [55]:
pca = PCA(n_components = dimension_chosen_rev)

In [56]:
X_train_rev_pca = pca.fit_transform(X_train_rev)
X_test_rev_pca = pca.fit_transform(X_test_rev)

In [57]:
print("X_train_pca_rev shape is : ", X_train_rev_pca.shape)
print("X_test_pca_rev shape  is : ", X_test_rev_pca.shape)
print("y_train shape is : ", y_train_cat.shape)
print("y_test shape is : ", y_test_cat.shape)

X_train_pca_rev shape is :  (80000, 56)
X_test_pca_rev shape  is :  (20000, 56)
y_train shape is :  (80000,)
y_test shape is :  (20000,)


## 2. Learning Simple Classifiers

* Choose **`X` classifiers** (https://scikit-learn.org/stable/supervised_learning.html#supervised-learning).
* Use **grid-search and stratified 10 fold cross-validation** to estimate the best parameters (https://scikit-learn.org/stable/model_selection.html#model-selection). 
* Present mean and standard deviation of accuracy, precision and recall.
* Show confusion matrices.

In [271]:
scores = ['accuracy','precision','recall'] #we will print out the mean and std for these 3 metrics for each parameter combination

In [272]:
kfolds = StratifiedKFold(10) #using stratified 10 fold cross-validation

> - For all the classification models, we will use **accuracy** as the scoring metric parameter in the GridSearchCV.

> - **2.1. Logistic Regression**

In [325]:
tuned_parameters= dict(max_iter=[60, 80, 100, 120],  #choosing a parameter grid for the gridsearch
                       C=[1,2,3], 
                       penalty=['l1','l2','elasticnet','none'])

lr_clf = GridSearchCV(LogisticRegression(solver='saga'), #using gridsearch
                   tuned_parameters, 
                   scoring='accuracy', 
                   cv=kfolds.split(X_train_cat_pca, y_train_cat))

lr_clf.fit(X_train_cat_pca, y_train_cat)

y_true, y_pred = y_test_cat, lr_clf.predict(X_test_cat_pca)

print("Best parameters:",lr_clf.cv_results_['params'][lr_clf.best_index_])
print()
print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred))
print()

print('Accuracy', np.round(accuracy_score(y_true,y_pred),3), #presenting the mean and standard deviation of accuracy
      '( mean:', 
       np.round(lr_clf.cv_results_['mean_test_score'][lr_clf.best_index_],3), 
       '& std:',
       np.round(lr_clf.cv_results_['std_test_score'][lr_clf.best_index_],3),
      ')')

print('Recall', np.round(recall_score(y_true,y_pred, average='weighted'),3)) #getting the recall score
print('Precision', np.round(precision_score(y_true,y_pred, average='weighted'),3)) #getting the precision score

Best parameters: {'C': 1, 'max_iter': 60, 'penalty': 'l1'}

Confusion matrix:
[[5967 2281  541]
 [2177 6503    0]
 [ 873   24 1634]]

Accuracy 0.705 ( mean: 0.75 & std: 0.005 )
Recall 0.705
Precision 0.706


> - **2.2. Decision Tree Classifier**

In [326]:
#using the same structure of the code in 2.1. for all the classification models
tuned_parameters= dict(criterion=['entropy','gini'],
                       max_depth= [4,6,8,10,15,20],
                      min_samples_split = [2,3,4,5,6,7,8,9,10])

dt_clf = GridSearchCV(
        DecisionTreeClassifier(), 
        tuned_parameters, 
        scoring='accuracy',
        cv=kfolds.split(X_train_cat_pca, y_train_cat))
    
dt_clf.fit(X_train_cat_pca, y_train_cat)

y_true, y_pred = y_test_cat, dt_clf.predict(X_test_cat_pca)

print("Best parameters:",dt_clf.cv_results_['params'][dt_clf.best_index_])
print()
print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred))
print()

print('Accuracy', np.round(accuracy_score(y_true,y_pred),3), 
      '( mean:', 
       np.round(dt_clf.cv_results_['mean_test_score'][dt_clf.best_index_],3), 
       '& std:',
       np.round(dt_clf.cv_results_['std_test_score'][dt_clf.best_index_],3),
      ')')

print('Recall', np.round(recall_score(y_true,y_pred, average='weighted'),3))
print('Precision', np.round(precision_score(y_true,y_pred, average='weighted'),3))

Best parameters: {'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 3}

Confusion matrix:
[[5748 2258  783]
 [2562 6104   14]
 [ 801   78 1652]]

Accuracy 0.675 ( mean: 0.673 & std: 0.005 )
Recall 0.675
Precision 0.676


> - **1.3. SVC**

In [327]:
#using the same structure of the code in 2.1. for all the classification models
tuned_parameters= dict(kernel=['rbf','linear'],
                       C= [1,2,3])

svc_clf = GridSearchCV(
        SVC(), 
        tuned_parameters, 
        scoring='accuracy',
        cv=kfolds.split(X_train_cat_pca, y_train_cat))
    
svc_clf.fit(X_train_cat_pca, y_train_cat)

y_true, y_pred = y_test_cat, svc_clf.predict(X_test_cat_pca)

print("Best parameters:",svc_clf.cv_results_['params'][svc_clf.best_index_])
print()
print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred))
print()

print('Accuracy', np.round(accuracy_score(y_true,y_pred),3), 
      'mean:', 
       np.round(svc_clf.cv_results_['mean_test_score'][svc_clf.best_index_],3), 
       '& std:',
       np.round(svc_clf.cv_results_['std_test_score'][svc_clf.best_index_],3),
      ')')

print('Recall', np.round(recall_score(y_true,y_pred, average='weighted'),3))
print('Precision', np.round(precision_score(y_true,y_pred, average='weighted'),3))

Best parameters: {'C': 3, 'kernel': 'rbf'}

Confusion matrix:
[[6270 1964  555]
 [2229 6446    5]
 [ 889   23 1619]]

Accuracy 0.717 mean: 0.812 & std: 0.005 )
Recall 0.717
Precision 0.719


## 3. Learning Simple Regressors

* Choose **`X` regressors** (https://scikit-learn.org/stable/supervised_learning.html#supervised-learning).
* Use **grid-search and 10 fold cross-validation** to estimate the best parameters (https://scikit-learn.org/stable/model_selection.html#model-selection). 
* Use the mean absolute error regression loss, or other relevant metrics.

> - We have chosen to analyse **R squared** as our performance metric for the regression models in order to compare the simple regressors with the ensemble models in task 4.

>   - **3.1. Lasso Regressor**

In [246]:
tuned_parameters= dict(alpha = [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], #defining parameter grid
                       max_iter = [100,200,500,1000],
                       selection = ["cyclic", "random"])

lasso_rg = GridSearchCV(linear_model.Lasso(), #using grid search
                       tuned_parameters,
                       cv=10, #using 10 fold CV
                       scoring='r2') #using r2 metric

lasso_rg.fit(X_train_rev_pca, y_train_rev)

print('Best parameters:', lasso_rg.best_params_) #getting best paramaters
print('R squared:', np.round(lasso_rg.best_score_,3))

Best parameters: {'alpha': 0.1, 'max_iter': 500, 'selection': 'random'}
R squared: 0.791


>   - **3.2. SVM Regressor**

In [258]:
#using the same structure of the code in 3.1. for all the regression models
tuned_parameters= dict(kernel=['rbf','linear'],
                       C= [1, 2,3],
                      degree= [2, 3,4,5,6])


svr_rg = GridSearchCV(SVR(), 
                       tuned_parameters,
                       cv=10,
                       scoring='r2')

svr_rg.fit(X_train_rev_pca, y_train_rev)

print('Best parameters:', svr_rg.best_params_)
print('R squared:', np.round(svr_rg.best_score_,3))

Best parameters: {'C': 2, 'degree': 2, 'kernel': 'linear'}
R squared: 0.478


In [259]:
#using the same structure of the code in 3.1. for all the regression models
tuned_parameters= dict(criterion=['mse','mae'],
                       max_depth= np.arange(2, 15),
                       min_samples_split = np.arange(2, 15))

dt_rg = GridSearchCV(DecisionTreeRegressor(), 
                       tuned_parameters,
                       cv=10,
                       scoring='r2')

dt_rg.fit(X_train_rev_pca, y_train_rev)

print('Best parameters:', dt_rg.best_params_)
print('R squared:', np.round(dt_rg.best_score_,3))

Best parameters: {'criterion': 'mse', 'max_depth': 4, 'min_samples_split': 2}
R squared: 0.555


## 4. Ensemble Learning

### 4.1. Voting Classifier/Regressor

* Use a voting classifier (http://scikit-learn.org/stable/modules/ensemble.html#voting-classifier)/regressor(https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html) to combine the best results of the `X` classifiers/regressors from previous sections. 

>   - **4.1.a) Voting Classifier**

In [61]:
clf1 = LogisticRegression(lr_clf.cv_results_['params'][lr_clf.best_index_])
clf2 = DecisionTreeClassifier(dt_clf.cv_results_['params'][dt_clf.best_index_])
clf3 = SVC(svc_clf.cv_results_['params'][svc_clf.best_index_])

voting_class_hard = VotingClassifier(estimators=[('logr', clf1), 
                                     ('rf', clf2), 
                                     ('svc', clf3)], 
                                     voting='hard')

voting_class_hard = voting_class_hard.fit(X_train_cat_pca, y_train_cat)

y_true, y_pred = y_test_cat, voting_class_hard.predict(X_test_cat_pca) 

print('Accuracy', np.round(accuracy_score(y_true,y_pred),3))
print('Recall', np.round(recall_score(y_true,y_pred, average='weighted'),3))
print('Precision', np.round(precision_score(y_true,y_pred, average='weighted'),3))

Accuracy 0.58
Recall 0.58
Precision 0.57


>   - **4.1.b) Voting Regressor**

In [241]:
rg1 = linear_model.Lasso(lasso_rg.best_params_)
rg2 = SVR(svr_rg.best_params_)
rg3 = DecisionTreeRegressor(dt_rg.best_params_)


voting_rg = VotingRegressor(estimators=[('lasso',rg1), 
                                     ('svr', rg2), 
                                     ('dtrg', rg3)])
voting_rg = voting_rg.fit(X_train_rev_pca, y_train_rev)

print('Best score:', np.round(voting_rg.score(X_test_rev_pca, y_test_rev),2))

Best score: 0.57


### 4.2. XGBoost 

* Use [XGBoost](https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn).

>  - **4.2.a) XGBoostClassifier**

In [263]:
#using merror (mlticlass classification error rate) as the evaluation metric to fit xgboost
xgb_clf = xgb.XGBClassifier(n_jobs=1, eval_metric='merror').fit(X_train_cat_pca, y_train_cat)

y_true, y_pred = y_test_cat, xgb_clf.predict(X_test_cat_pca)

print('Accuracy', np.round(accuracy_score(y_true,y_pred),3))
print('Recall', np.round(recall_score(y_true,y_pred, average='weighted'),3))
print('Precision', np.round(precision_score(y_true,y_pred, average='weighted'),3))

Accuracy 0.726
Recall 0.726
Precision 0.727


>  - **4.2.b) XGBoostRegressor**

In [264]:
xgb_rg = xgb.XGBRegressor(n_jobs=1, objectvie='reg:squarederror').fit(X_train_rev_pca, y_train)

y_true, y_pred = y_test_rev, xgb_rg.predict(X_test_rev_pca)

print('R squared: ', np.round(r2_score(y_true, y_pred),3))

R squared:  0.682


### 4.3. Random Forests

* Use [Random Forests](http://scikit-learn.org/stable/modules/ensemble.html#random-forests).

>  - **4.3.a) Random Forest Classifier**

In [268]:
tuned_parameters = {'n_estimators':[20,40,50,57],
                    'max_depth':[5,10,15,20],
                    'random_state':[42],
                    'criterion':['gini','entropy']}

clf = GridSearchCV(RandomForestClassifier(), 
                       tuned_parameters, 
                       scoring='accuracy',
                       cv=kfolds.split(X_train_cat_pca, y_train_cat))

clf.fit(X_train_cat_pca, y_train_cat)

y_true, y_pred = y_test_cat, clf.predict(X_test_cat_pca)
print()
print('Accuracy', np.round(accuracy_score(y_true,y_pred),3))
print('Recall', np.round(recall_score(y_true,y_pred, average='weighted'),3))
print('Precision', np.round(precision_score(y_true,y_pred, average='weighted'),3))

Accuracy 0.709
Recall 0.709
Precision 0.713


>  - **4.3.b)  Random Forest Regressor**

In [229]:
tuned_parameters = {'n_estimators':[20,40,50,57],
                    'max_depth':[5,10,15,20],
                    'random_state':[42],
                    'criterion':['mse','mae']}

rg = GridSearchCV(RandomForestRegressor(), 
                   tuned_parameters,
                   cv=10,
                   scoring='r2')

rg.fit(X_train_rev_pca, y_train_rev)

In [225]:
y_true, y_pred = y_test_rev, regr.predict(X_test_rev_pca)

print('R squared: ', np.round(r2_score(y_true, y_pred),3))

R squared:  0.604


## 5. Conclusion

Draw some final conclusions about this project work

> - For *task 1*, we discovered and visualized the data to gain some initial insights. We then prepared the data for our ML algorithms by tackling missing values (dropping some features and replacing some missing missing values with the mean) and by applying feature selection (creating dummy variables). Since we were asked to perform both classification and regression tasks, we created a new target feature revenue_category and used the stratified sampling method to ensure that the right number of instances are sampled from each of the 3 categories (<12K, 12K-20K, >20K) such that the test set was representative of the overall population. 
> - Lastly, we split the dataset into multiple subsets (training set 80% and test set 20%) and due to the high number of features present we applied PCA for both classification and regression data, being left with (X_train_cat_pca, y_train_cat, X_test_cat_pca, y_test_cat) and (X_train_rev_pca, y_train_rev, X_test_rev_pca, y_test_rev) respectively.

> - For *task 2*, we selected the following models: **logistic regression, svc** and **decision tree classifier**. We also selected accuracy, recall and precision as our performance measures for the classification models. We also got the confusion matrix for each one of them alongside the best parameter combination (that yielded the highest accuracy score). The model that performed best was the decision tree classifier, with an accuracy score of 0.717, behing the logistic regression (0.705) and the svc (0.675).

> - For *task 3*, we selected the following models: **lasso regression, svr** and **decision tree regresor**. We also selected R squared as our performance measure for the regression models. The model that performed best was the lasso regressor, with a R-squared score of 0.791, behing the decision tree (0.555) and the svr (0.478).

> - For *task 4*, in terms of the classification problem, we explored the hard voting method for our classification voting ensemble which involves summing the predictions for each class label and predicting the class label with the most votes. The voting classifier performed worse when compared to the individual models, only achieving an accuracy equal to 0.58, which is slightly better than random guessing. For further work, to potentially improve the performance of the voting classifier, we think it could be worth to weight the contribution of each of the single classification models to the final ensble, since we know that the logistic regression and the decision tree classifier are performing relatively well (with an accuracy 0.705 and 0.717 respectively). 
> - Next, we trained an xgboost classifier model, which is an ensemble model that can combine several predictors that are trained sequentially, each trying to correct the residual errors made by the previous predictor. We used all the default parameters, including the tree_method one in which xgboost will choose the most conservative option available. We only chose merror (mlticlass classification error rate) as the evaluation metric to fit xgboost. The **accuracy score** was **0.726**, as such, **xgboost classifier outperformed all the individual classifiers as well as the voting classifier**. For time saving purposes we did not tweak any further parameters but we thought it would be important to point out that we could further analyse the model's performance with the early_stopping_round, learning_rate, n_estimators and max_depth parameters changed. 
> - Lastly, we also used the random forest classifier model trained using the bagging method. While the dt model searches for the best feature when splitting a node, the rf model searches for the best feature among a random subset of features. The model's accuracy score was 0.709.

> - For *task 4*, in terms of the regression problem, we trained our voting regressor with the models from task 3 and the predictions we got were the average of the simple regressors. With an R squared of 0.57, the voting regressor achieved a better performance when compared to the svr (R2=0.478) and the dt (R2=0.555) models. The xgboost regressor's and random forest's R2 score was 0.682 and 0.604, respectively. Overall, the model that ended up **performed the best was the lasso regressor** **(R2=0.791)**, a regularized version of the linear regression model, which performs automatically feature selection. This could potentially point out the fact that even after performing PCA, our data still had least important features which were elimated by the L1 penalty.